In [44]:
import pandas as pd
import numpy as np
import uuid

In [45]:
df = pd.read_csv("missing_fight_ids.csv", header=None, names=range(231))
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)
df

"(index, fight_id)",fighter,event,date,opponent,timestep-1,timestep-2,timestep-3,timestep-4,timestep-5,timestep-6,...,timestep-218,timestep-219,timestep-220,timestep-221,timestep-222,timestep-223,timestep-224,timestep-225,timestep-226,timestep-227
0,Daichi Abe,UFC Fight Night 132: Cowboy vs. Edwards,2018-06-23,Li Jingliang,2018-06-13 12:09:00 - price: +240,2018-06-13 12:21:00 - price: +325,2018-06-13 13:42:00 - price: +315,2018-06-13 15:24:00 - price: +323,2018-06-13 16:18:00 - price: +320,2018-06-13 17:06:00 - price: +313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mansur Abdul-Malik,UFC,2024-11-10,Dusko Todorovic,2024-10-27 17:16:00 - price: -360,2024-10-29 04:14:00 - price: -355,2024-10-30 12:34:00 - price: -366,2024-10-30 17:52:00 - price: -368,2024-11-01 18:04:00 - price: -362,2024-11-02 14:18:00 - price: -360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nick Aguirre,UFC Fight Night 218,2023-01-14,Dan Arguetta,2023-01-10 16:14:00 - price: +285,2023-01-10 16:20:00 - price: +375,2023-01-10 16:48:00 - price: +375,2023-01-10 18:12:00 - price: +375,2023-01-10 19:56:00 - price: +379,2023-01-10 20:16:00 - price: +391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amir Albazi,UFC on ESPN: Hermansson vs. Allen,2023-06-03,Kai Kara-France,2023-05-09 19:46:00 - price: +175,2023-05-09 20:22:00 - price: -172,2023-05-09 23:24:00 - price: -180,2023-05-10 00:08:00 - price: -177,2023-05-10 06:02:00 - price: -170,2023-05-10 14:44:00 - price: -155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Junior Albini,UFC 224: Nunes vs. Pennington,2018-05-12,Aleksei Oliynyk,2018-04-23 15:15:00 - price: -120,2018-04-23 15:21:00 - price: -110,2018-04-23 18:33:00 - price: -108,2018-04-23 18:48:00 - price: -103,2018-04-23 19:57:00 - price: -103,2018-04-23 21:45:00 - price: -115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,Elizeu Zaleski dos Santos,UFC on ESPN+ 15: Andrade vs. Zhang,2019-08-31,Li Jingliang,2019-08-25 16:57:00 - price: -285,2019-08-25 17:09:00 - price: -350,2019-08-25 17:42:00 - price: -305,2019-08-25 18:48:00 - price: -285,2019-08-25 20:54:00 - price: -305,2019-08-26 01:54:00 - price: -310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1535,Robert Whittaker,UFC Fight Night 27: Condit vs. Kampmann II,2013-08-28,Court McGee,2013-08-14 20:18:00 - price: -130,2013-08-15 00:09:00 - price: -165,2013-08-16 19:15:00 - price: -175,2013-08-18 03:51:00 - price: -155,2013-08-18 03:57:00 - price: -140,2013-08-18 05:12:00 - price: -140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1536,Zhalgas Zhumagulov,UFC Fight Night: Font vs. Aldo,2021-12-04,Manel Kape,2021-11-09 17:34:00 - price: +220,2021-11-09 17:40:00 - price: +185,2021-11-10 02:00:00 - price: +190,2021-11-10 08:06:00 - price: +191,2021-11-10 08:08:00 - price: +190,2021-11-10 14:47:00 - price: +180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1537,Youssef Zalal,UFC Fight Night 184: Overeem vs. Volkov,2021-02-06,Seung Woo Choi,2021-01-28 03:46:00 - price: -150,2021-01-28 04:36:00 - price: -155,2021-01-28 10:08:00 - price: -170,2021-01-28 10:12:00 - price: -190,2021-01-28 19:52:00 - price: -199,2021-01-28 19:54:00 - price: -202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
timestamp_cols = [col for col in df.columns if col.startswith('timestep-')]
df_clean = df.copy()

# Apply split to each timestamp column before converting to NumPy array
for col in timestamp_cols:
    df_clean[col] = df_clean[col].astype(str).str.split(' - price: ').str[0]

# Now precompute the timestamp sets using cleaned columns
timestamps_matrix = df_clean[timestamp_cols].to_numpy()
timestamp_sets = [set(row) for row in timestamps_matrix]

for i, current_set in enumerate(timestamp_sets):
    row = df_clean.iloc[i]
    comparisons = timestamp_sets[:i] + timestamp_sets[i+1:]
    indices = list(range(i)) + list(range(i+1, len(df_clean)))
    
    match_counts = [len(current_set & other_set) for other_set in comparisons]
    matched_indices = [indices[j] for j, count in enumerate(match_counts) if count >= 20]

    if matched_indices:
        for match_index in matched_indices:
            matched_row = df_clean.iloc[match_index]
            shared_count = len(current_set & timestamp_sets[match_index])

            print(f"Match found:")
            print(f" - Current: fighter = {row['fighter']}, opponent = {row['opponent']}")
            print(f" - Match:  fighter = {matched_row['fighter']}, opponent = {matched_row['opponent']}")
            print(f" - Shared timestamps = {shared_count}\n")
            print(f"-----------------FIX CODE------------------")
            print(f"df['fighter'] = df['fighter'].replace('{matched_row['fighter']}', '{row['fighter']}')")
            print(f"df['opponent'] = df['opponent'].replace('{matched_row['fighter']}', '{row['fighter']}')")
            print(f"-------------FIX CODE OTHER-------------")
            print(f"df['fighter'] = df['fighter'].replace('{matched_row['opponent']}', '{row['opponent']}')")
            print(f"df['opponent'] = df['opponent'].replace('{matched_row['opponent']}', '{row['opponent']}')")
            print(f"-------------FIX CODE OTHER-------------")
            print(f"df['fighter'] = df['fighter'].replace('{matched_row['fighter']}', '{row['opponent']}')")
            print(f"df['opponent'] = df['opponent'].replace('{matched_row['fighter']}', '{row['opponent']}')")
            print("\n\n\n")

Match found:
 - Current: fighter = Mayra Bueno Silva, opponent = Yanan Wu
 - Match:  fighter = Pat Sabatini, opponent = TJ Laramie
 - Shared timestamps = 23

-----------------FIX CODE------------------
df['fighter'] = df['fighter'].replace('Pat Sabatini', 'Mayra Bueno Silva')
df['opponent'] = df['opponent'].replace('Pat Sabatini', 'Mayra Bueno Silva')
-------------FIX CODE OTHER-------------
df['fighter'] = df['fighter'].replace('TJ Laramie', 'Yanan Wu')
df['opponent'] = df['opponent'].replace('TJ Laramie', 'Yanan Wu')
-------------FIX CODE OTHER-------------
df['fighter'] = df['fighter'].replace('Pat Sabatini', 'Yanan Wu')
df['opponent'] = df['opponent'].replace('Pat Sabatini', 'Yanan Wu')




Match found:
 - Current: fighter = Shamil Gamzatov, opponent = Michar Oleksiejczuk
 - Match:  fighter = Lerone Murphy, opponent = Makwan Amirkhani
 - Shared timestamps = 21

-----------------FIX CODE------------------
df['fighter'] = df['fighter'].replace('Lerone Murphy', 'Shamil Gamzatov')
df['

In [ ]:
df_clean = df.copy()

for index, row in df.iterrows():
    